### Side note:  Ik heb nu alleen de input en output omgezet naar onze situatie.

### Alle transformaties rommel is nu gedaan door onze grootste buddy chatgpt

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Bitcoin Batch Pipeline")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")
# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()
# Load data from BigQuery.
df = spark.read \
  .format("bigquery") \
  .load("data-engineering-labs-guido.labdataset.bitcoin_data")    # project_id.datatset.tablename. Use your project id
df.printSchema()
df.show(4)

In [ ]:
# Verwijder missende waarden
cleaned_data = df.na.drop()

# Controleer op negatieve prijzen of volumes
cleaned_data = cleaned_data.filter((cleaned_data['Open'] >= 0) & 
                                   (cleaned_data['Price'] >= 0) & 
                                   (cleaned_data['Vol_'] >= 0))

# Toon een voorbeeld van de opgeschoonde data
cleaned_data.show(5)


In [ ]:
from pyspark.sql.functions import col, expr

# Bereken wekelijkse prijsverandering in procenten
transformed_data = cleaned_data.withColumn(
    "Weekly_Change", ((col("Close") - col("Open")) / col("Open")) * 100
)

# Bereken volatiliteit
transformed_data = transformed_data.withColumn(
    "Volatility", col("High") - col("Low")
)

# Toon de getransformeerde data
transformed_data.show(5)


In [ ]:
from pyspark.sql.functions import month, year, avg, sum, max

# Voeg maand- en jaarkolommen toe
transformed_data = transformed_data.withColumn("Month", month(col("Date"))) \
                                   .withColumn("Year", year(col("Date")))

# Bereken maandelijkse aggregaties
monthly_data = transformed_data.groupBy("Year", "Month") \
    .agg(
        avg("Close").alias("Avg_Close"),
        sum("Volume").alias("Total_Volume"),
        max("Volatility").alias("Max_Volatility")
    )

# Toon maandelijkse resultaten
monthly_data.show(5)


In [ ]:
# Use the Cloud Storage bucket for temporary BigQuery export data used by the connector.
bucket = "de_jads_tempguido"  # use your bucket 
spark.conf.set('temporaryGcsBucket', bucket)
# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
# Saving the data to BigQuery
ordered_word_count.write.format('bigquery') \
  .option('table', 'data-engineering-labs-guido.labdataset.btc_version1') \
  .mode("append") \
  .save()

In [ ]:
# Stop the spark context
spark.stop()